# Перенос обучения и тонкая настройка обученных моделей

В этом руководстве вы узнаете, как классифицировать изображения кошек и собак с помощью трансферного обучения из предварительно обученной сети.

Предварительно обученная модель — это сохраненная сеть, которая ранее была обучена на большом наборе данных, как правило, на крупномасштабной задаче классификации изображений. Вы либо используете предварительно обученную модель как есть, либо используете трансферное обучение, чтобы настроить эту модель для данной задачи.

Интуиция, лежащая в основе трансферного обучения для классификации изображений, заключается в том, что если модель обучается на большом и достаточно общем наборе данных, эта модель будет эффективно служить общей моделью визуального мира. Затем вы можете воспользоваться этими изученными картами объектов, не начав с нуля, обучая большую модель на большом наборе данных.

В этой записной книжке вы попробуете два способа настройки предварительно обученной модели:

1. Извлечение признаков: используйте представления, изученные предыдущей сетью, для извлечения значимых признаков из новых образцов. Вы просто добавляете новый классификатор, который будет обучаться с нуля, поверх предварительно обученной модели, чтобы вы могли повторно использовать карты объектов, изученные ранее, для набора данных.

1. Вам не нужно (повторно) обучать всю модель. Базовая сверточная сеть уже содержит функции, которые обычно полезны для классификации изображений. Однако окончательная классификационная часть предварительно обученной модели специфична для исходной задачи классификации, а затем специфична для набора классов, на которых была обучена модель.

1. Тонкая настройка: разморозьте несколько верхних слоев замороженной базовой модели и совместно обучите как недавно добавленные слои классификатора, так и последние слои базовой модели. Это позволяет нам «точно настроить» представления функций более высокого порядка в базовой модели, чтобы сделать их более подходящими для конкретной задачи.

Вы будете следовать общему рабочему процессу машинного обучения.

1. Изучите и поймите данные
1. Создайте конвейер ввода, в данном случае с помощью Keras ImageDataGenerator.
1. Составьте модель
 - Загрузить предварительно обученную базовую модель (и предварительно обученные веса)
 - Подключите классификационные слои сверху
1. Обучите модель
1. Оцените модель

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

## Обработка данных

### Загрузка данных

В этом уроке мы будем использовать набор данных, содержащий несколько тысяч изображений кошек и собак. Загрузите и извлеките zip-файл, содержащий изображения, затем создайте файл `tf.data.Dataset` для обучения и проверки с помощью утилиты `tf.keras.utils.image_dataset_from_directory`. Вы можете узнать больше о загрузке изображений в дополнительном [учебном пособии](https://www.tensorflow.org/tutorials/load_data/images).

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

In [ ]:
PATH

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Покажите первые девять изображений и меток из обучающего набора:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Поскольку исходный набор данных не содержит тестового набора, вы создадите его. Для этого определите, сколько пакетов данных доступно в наборе проверки, используя `tf.data.experimental.cardinality`, затем переместите 20% из них в тестовый набор.

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [ ]:
val_batches

In [ ]:
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

## Задание 1.
Загрузить набор данных `tf_flowers`, сформировать на нем набор данных подготовить его к загрузке в модель **ModelNet_v2**. Набор данных необходимо разбить на пакеты по 32 изображений. Отобразить случайные 9 изображений из валидационной выборки.

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

IMG_SIZE = 160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 255.0)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

# Разбиваем на пакеты
BATCH_SIZE = 32
train_batches = train.batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

# Отображаем 9 случайных изображений из валидационной выборки
plt.figure(figsize=(10, 10))
for images, labels in validation_batches.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(metadata.features['label'].int2str(labels[i]))
        plt.axis('off')
plt.show()

### Настройте набор данных для повышения производительности

Используйте предварительную выборку с буферизацией для загрузки изображений с диска без блокировки ввода-вывода. Чтобы узнать больше об этом методе, смотрите раздел [производительность данных](https://www.tensorflow.org/guide/data_performance) путеводитель.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Используем расширение набора данных

Если у вас нет большого набора данных изображений, рекомендуется искусственно вносить разнообразие в выборку, применяя случайные, но реалистичные преобразования к обучающим изображениям, такие как поворот и горизонтальное переворачивание. Это помогает предоставить модели доступ к различным аспектам обучающих данных и уменьшить [переобучение](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). Вы можете узнать больше об увеличении данных в этом [руководстве](https://www.tensorflow.org/tutorials/images/data_augmentation).

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

Примечание: Эти слои активны только во время тренировки, когда вы вызываете `Model.fit`. Они неактивны, когда модель используется в режиме логического вывода в `Model.evaluate`, `Model.predict` или `Model.call`.

Давайте повторно применим эти слои к одному и тому же изображению и посмотрим на результат.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Масштабировать значения пикселей

Загрузите `tf.keras.applications.MobileNetV2` для использования в качестве базовой модели. Эта модель ожидает значения пикселей в `[-1, 1]`, но на данный момент значения пикселей в ваших изображениях находятся в `[0, 255]`. Чтобы изменить их масштаб, используйте метод предварительной обработки, прилагаемый к модели.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

Примечание: В качестве альтернативы вы могли бы изменить масштаб значений пикселей с `[0, 255]` на `[-1, 1]`, используя `tf.keras.layers.Rescaling`.

In [ ]:
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

Примечание: Если вы используете другие `tf.keras.applications`, обязательно проверьте документ API, чтобы определить, ожидают ли они пиксели в `[-1, 1]` или `[0, 1]`, или используйте включенную функцию `preprocess_input`.

## Задание 2
Используйте расширение набора данных, созданного в задании 1, с использованием конвеера преобразования `data_augmentation`(см. ячейки выше), с включением в него слоя RandomCrop() (возможно придется реализовать слой восстанавливающий разрешение изображения после обрезки, а возможно и нет, исследуйте этот вопрос).

In [ ]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


train_batches = raw_train.map(preprocess_image).batch(BATCH_SIZE)
validation_batches = raw_validation.map(preprocess_image).batch(BATCH_SIZE)


plt.figure(figsize=(10, 10))
for images, _ in validation_batches.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.axis('off')
plt.show()

plt.figure(figsize=(10, 10))
for images, _ in validation_batches.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(images[i], 0), training=True)
        plt.imshow(augmented_image[0].numpy())
        plt.axis('off')
plt.show()

In [ ]:
for images, _ in train_batches.take(1):
    for i in range(1):
        original_image = images[i]
        cropped_image = data_augmentation(tf.expand_dims(original_image, 0), training=True)

        print("Оригинальный размер: ", original_image.numpy().shape)
        print("Размер после RandomCrop: ", cropped_image.numpy().shape)

In [ ]:
#реализовывать слой восстановления не нужно

## Создайте базовую модель из предварительно обученных CNN
Вы создадите базовую модель на основе модели **MobileNet V2**, разработанной в Google. Она предварительно обучена на наборе данных ImageNet, большом наборе данных, состоящем из 1,4 млн изображений и 1000 классов. ImageNet - это исследовательский обучающий набор данных с широким спектром категорий, таких как `джекфрут` и `шприц`. Эта база знаний поможет нам классифицировать кошек и собак из нашего конкретного набора данных.

Во-первых, вам нужно выбрать, какой уровень MobileNet V2 вы будете использовать для извлечения объектов. Самый последний уровень классификации, т. е. *голова*("сверху", поскольку большинство диаграмм моделей машинного обучения идут снизу вверх) не очень полезен. Вместо этого вы будете следовать общепринятой практике и использовать самый последний слой перед операцией уплощения. Этот слой называется "слоем узкого места". Элементы слоя "узкое место" сохраняют большую общность по сравнению с конечным/верхним слоем.

Сначала создайте экземпляр модели MobileNet V2, предварительно загруженной с весами, обученными в ImageNet. Указав аргумент **include_top=False**, вы загружаете сеть, которая не включает слои классификации вверху, что идеально подходит для извлечения объектов.

In [ ]:
IMG_SIZE = (160, 160)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Этот инструмент извлечения объектов преобразует каждое изображение размером `160x160x3` в блок объектов размером `5x5x1280`. Давайте посмотрим, что это делает с примером пакета изображений:

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Извлечение объектов
На этом шаге вы заморозите сверточную базу, созданную на предыдущем шаге, и будете использовать ее в качестве средства извлечения объектов. Кроме того, вы добавляете поверх него классификатор и обучаете классификатор верхнего уровня.

### Заморозка сверточной части нейронной сети

Важно заморозить сверточную часть перед компиляцией и обучением модели. Замораживание (путем установки layer.trainable = False) предотвращает обновление весов в данном слое во время обучения. Mobile Net V2 имеет много уровней, поэтому установка флага `trainable` для всей модели в значение False приведет к замораживанию всех из них.

In [ ]:
base_model.trainable = False

### Важное замечание о слоях пакетной нормализации

Многие модели содержат `tf.keras.layers.BatchNormalization`. Этот слой является особым случаем, и при его точной настройке следует соблюдать меры предосторожности, как показано далее в этом руководстве.

Когда вы устанавливаете `layer.trainable = False`, слой `BatchNormalization` будет работать в режиме логического вывода и не будет обновлять статистику среднего значения и дисперсии.

Когда вы размораживаете модель, содержащую слои пакетной нормализации, для выполнения точной настройки, вы должны поддерживать слои пакетной нормализации в режиме логического вывода, передавая `training = False` при вызове базовой модели. В противном случае обновления, примененные к неподготовляемым весам, уничтожат то, чему научилась модель.

Для получения более подробной информации смотрите [Руководство по обучению переводу](https://www.tensorflow.org/guide/keras/transfer_learning).

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Добавление *головы* - классификатора

Чтобы сгенерировать прогнозы на основе блока объектов, усредните их по пространственным местоположениям размером `5x5`, используя `tf.keras.layers.GlobalAveragePooling2D` для преобразования объектов в один вектор из 1280 элементов на изображение.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

Примените `tf.keras.layers.Dense` слой для преобразования этих объектов в единый прогноз для каждого изображения. Здесь вам не нужна функция активации, потому что это предсказание будет обрабатываться как `logit`, или необработанное значение прогноза. Положительные числа предсказывают класс 1, отрицательные числа предсказывают класс 0.

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Создайте модель, объединив в цепочку слои увеличения данных, масштабирования, `base_model` и извлечения объектов с помощью [Keras Functional API](https://www.tensorflow.org/guide/keras/functional). Как упоминалось ранее, используйте `training=False`, поскольку наша модель содержит слой `BatchNormalization`.

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

Более 8 миллионов параметров в мобильной сети заморожены, но в плотном слое есть 1,2 тысячи *обучаемых* параметров. Они разделены между двумя объектами `tf.Variable`, весами и смещениями.

In [ ]:
len(model.trainable_variables)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

## Задание 3.
Настройте классификатор для модели **MobileNet_v2** для классификации набора данных `tf_flowers`. Проверьте настройку по summary и визуализируйте модель.

In [ ]:
IMG_SHAPE = (160, 160, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(5, activation='softmax')

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

tf.keras.utils.plot_model(model, show_shapes=True)

### Скомпилируйте модель

Скомпилируйте модель перед ее обучением. Поскольку существует два класса, используйте `tf.keras.losses.BinaryCrossentropy` как функцию ошибки с флагом `from_logits=True`, поскольку модель обеспечивает линейный вывод.

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])

### Обучите модель

После обучения в течение 10 эпох вы должны увидеть точность ~ 96% в наборе проверки.

In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_batches)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

## Задание 4.
Обучите модель и постройте ее истории обучения на валидационной и обучающей выборках.

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])
initial_epochs = 10
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))
loss0, accuracy0 = model.evaluate(validation_batches)
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

In [ ]:
EPOCHS = 10
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Кривые истории обучения

Давайте взглянем на кривые обучения и потери точности при проверке при использовании базовой модели MobileNetV2 в качестве средства извлечения фиксированных функций.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Примечание: Если вам интересно, почему показатели проверки явно лучше, чем показатели обучения, основной фактор заключается в том, что такие слои, как `tf.keras.layers.BatchNormalization` и `tf.keras.layers.Dropout` влияет на точность во время обучения. Они отключаются при расчете потерь при проверке.

В меньшей степени это связано также с тем, что показатели обучения сообщают среднее значение за эпоху, в то время как показатели проверки оцениваются после эпохи, поэтому показатели проверки показывают модель, которая обучалась немного дольше.

## Точная настройка
В эксперименте по извлечению объектов вы обучали только несколько слоев поверх базовой модели MobileNetV2. Веса предварительно обученной сети **не** обновлялись во время обучения.

Один из способов еще больше повысить производительность - это обучить (или "точно настроить") веса верхних слоев предварительно обученной модели наряду с обучением добавленного вами классификатора. Процесс обучения приведет к принудительной настройке весов общих признаков на признаки, связанные конкретно с набором данных.

Примечание: Это следует делать только после того, как вы обучили классификатор верхнего уровня с предварительно обученной моделью, для которой установлено значение non-trainable. Если вы добавите случайно инициализированный классификатор поверх предварительно обученной модели и попытаетесь обучить все слои совместно, величина обновлений градиента будет слишком большой (из-за случайных весов из классификатора), и ваша предварительно обученная модель забудет то, чему она научилась.

Кроме того, вам следует попытаться точно настроить небольшое количество верхних слоев, а не всю модель MobileNet. В большинстве сверточных сетей, чем выше уровень, тем более специализированным он является. На первых нескольких слоях изучаются очень простые и универсальные признаки, которые применимы практически ко всем типам изображений. По мере продвижения вверх признаки становятся все более специфичными для набора данных, на основе которого была обучена модель. Цель тонкой настройки состоит в том, чтобы адаптировать эти специализированные признаки для работы с новым набором данных, а не перезаписывать общее обучение.

### Разморозка верхних слоев модели


Все, что вам нужно сделать, это разморозить `base_model` и установить, что нижние слои не поддаются обучению. Затем вам следует перекомпилировать модель (необходимо, чтобы эти изменения вступили в силу) и возобновить обучение.

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Скомпилируйте модель

Поскольку вы тренируете гораздо более крупную модель и хотите заново адаптировать предварительно подготовленные веса, на данном этапе важно использовать более низкую скорость обучения. В противном случае ваша модель может очень быстро перестроиться.

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0, name='accuracy')])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Продолжение обучения

Если вы тренировались сходимости ранее, этот шаг повысит вашу точность на несколько процентных пунктов.

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Давайте взглянем на кривые обучения и потери точности при проверке при точной настройке последних нескольких уровней базовой модели MobileNetV2 и обучении классификатора поверх нее. Потери при проверке намного выше, чем при обучении, так что вы можете немного переобучиться.

Вы также можете немного переобучиться, поскольку новый обучающий набор относительно невелик и похож на исходные наборы данных MobileNetV2.

После точной настройки модель достигает почти 98% точности в наборе для проверки.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Оценка и прогнозирование

Наконец, вы можете проверить производительность модели на новых данных, используя тестовый набор.

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

И теперь вы готовы использовать эту модель, чтобы предсказать, является ли ваш питомец кошкой или собакой.

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## Задание 5
Разморозьте верхние 20 слоев модели **MobileNet_V2** и выполните дообучение модели из Заданий 3-4. Постройте  совместные кривые истории обучения. Визиализируйте предсказания на 9 изображениях из тестового набора данных.

## Краткое описание

* ** Использование предварительно обученной модели для извлечения признаков **: При работе с небольшим набором данных обычной практикой является использование преимуществ функций, изученных моделью, обученной на большем наборе данных в той же предметной области. Это делается путем создания экземпляра предварительно обученной модели и добавления сверху полностью подключенного классификатора. Предварительно обученная модель "заморожена", и во время обучения обновляются только веса классификатора.
В этом случае сверточная база извлекла все объекты, связанные с каждым изображением, и вы просто обучили классификатор, который определяет класс изображения с учетом этого набора извлеченных объектов.

* ** Точная настройка предварительно обученной модели **:
Для дальнейшего повышения производительности может потребоваться переназначить слои верхнего уровня предварительно обученных моделей для нового набора данных с помощью точной настройки. В этом случае вы настроили свои веса таким образом, чтобы ваша модель изучала высокоуровневые функции, специфичные для набора данных. Этот метод обычно рекомендуется, когда обучающий набор данных большой и очень похож на исходный набор данных, на котором была обучена предварительно обученная модель.

To learn more, visit the [Transfer learning guide](https://www.tensorflow.org/guide/keras/transfer_learning).
